# Project 1: Probing direction selectivity in the mouse retina

Welcome to the first project of the class. 

**You will learn to:** 
- Construct direction tuning curves from grating data.
- Quantify direction selectivity.
- Perform statistical comparison of paired samples.

Let's first import the packages we are going to use, and set up some plotting parameters.

In [ ]:
# import packages
import numpy as np
import matplotlib.pyplot as plt
import spkbasic
from os import listdir
from scipy import stats
from pathlib import Path

%matplotlib inline
plt.rcParams['figure.figsize'] = (15.0, 6.0) # set default size of plots

## 1 - Creating tuning curves

The stimulus times, as well as the spikes of 20 cells are in the folder data_drifting_grating. Let's first start by loading the stimulus times.

In [ ]:
datapath='data_drifting_grating/'
stimulus = np.loadtxt(datapath+'stimulus.txt')

Each cycle contains the presentations of gratings drifting in one of 8 directions, for 400 frames, with a period of 100 frames. There is a pulse in the beginning of each period, except for the first one (as it's usually ignored in the analysis). In between the directions, there is a gray screen of 300 frames.

Let's first input declare our values:

In [ ]:
duration=400
period=100
Nangles=8
Ncycles=4

Now we can calculate some imporant values that will help sort out the spikes relative to the stimulus times.

In [ ]:
angles=np.linspace(0,2*np.pi,num=8,endpoint=False) # the angles of the drifting grating in radians
NanglePulses=int(np.floor(duration/period))-1
NcyclePulses=NanglePulses*Nangles
NstimPulses=NcyclePulses*Ncycles
pulseTimes=stimulus[0:int(NstimPulses)]
pulseTimes=np.reshape(pulseTimes,(Ncycles,Nangles,NanglePulses))
periodDuration=np.mean(np.diff(pulseTimes,n=1,axis=2))

Having set up the stimulus times, now we can count spikes! Let's write a function that loads the spikes of an example cell:

In [ ]:
def calculateSpikeCounts(pulseTimes,spikeTimes):
    periodDuration=np.mean(np.diff(pulseTimes,n=1,axis=2)) # get period duration
    pulseTimesShape=np.shape(pulseTimes)
    spikeCounts=np.zeros(pulseTimesShape[:2]) # pre-allocate the array
    
    for iCycle in range(0,pulseTimesShape[0]):
        for iDirection in range(0,pulseTimesShape[1]):
            dirSpikes=spikeTimes[spikeTimes>=pulseTimes[iCycle,iDirection,0]]
            dirSpikes=dirSpikes[dirSpikes<pulseTimes[iCycle,iDirection,-1]+periodDuration]
            spikeCounts[iCycle,iDirection]=np.size(dirSpikes)
    return spikeCounts

Let's test our function below. Our goal to to reach to a matrix Ntrials x Ndirections, which we will use for most of our calculations...

In [ ]:
exampleSpikeTimes=np.loadtxt('data_drifting_grating/5_SP_C3601.txt')
exampleSpikeCounts=calculateSpikeCounts(pulseTimes,exampleSpikeTimes)
print(exampleSpikeCounts)

The first thing we can do is plot a tuning curve. We plot the spike count vs the direction

In [ ]:
tuningCurve=np.mean(exampleSpikeCounts,axis=0)
plt.plot(np.rad2deg(angles),tuningCurve)
plt.title('Tuning Curve')
plt.ylabel('Spike count');
plt.xlabel('Direction (deg)');

It is also customary to plot tuning curve in a polar plot.

**Exercise:** Plot the tuning curve in a polar plot by filling in ```plotTuningCurvePolar(angles, responses)```. Consider using ```plt.polar```, and try to make your plot pretty.

*Hint*: Make sure that your final curve is closed!

In [ ]:
def plotTuningCurvePolar(angles,responses):
    '''
    Inputs:
            angles: Ndirections np array
            responses: Ntrials x Ndirections np array
    '''
    
    ### START CODE HERE ### (approx. 1-2 lines)
    plt.polar(np.append(angles,angles[0]),np.append(responses,responses[0]),linewidth=2)
    ### END CODE HERE ###

In [ ]:
plotTuningCurvePolar(angles,tuningCurve)

We want now to look in all of our data. We will now load the spike trains of all cells in the dataset in a list, and you will have to figure out how to properly get structured spike counts for all cells now.

In [ ]:
filenames = [i.name for i in Path(datapath).glob('5_SP_C*.txt')]
allSpikeTrains=[np.loadtxt(Path(datapath,fpath)) for fpath in filenames] # list of spike trains

**Exercise:** Let's now calculate the tuning curves for all cells in our dataset. First, we need to sort the spikes of all cells in spike counts. Fill in the function that does that:

*Hint:* You can iterate over cells using the function ```calculateSpikeCounts```, but you can also start over!

In [ ]:
def calculateAllSpikeCounts(pulseTimes,allSpikeTimes):
    '''
    Inputs:
            angles: list of strings
            allSpikeTimes: list of spike trains (in np arrays)
    Outputs:
            allSpikeCounts:
    '''
    
    ### START CODE HERE ### (approx. 5-10 lines)
    Ncells=len(allSpikeTimes)
    Ncycles,Ndirs,_=np.shape(pulseTimes)
    allSpikeCounts=np.zeros((Ncells,Ncycles,Ndirs)) # pre-allocate the array  
    periodDuration=np.mean(np.diff(pulseTimes,n=1,axis=2)) # get period duration

    for iCycle in range(0,Ncycles):
        for iDirection in range(0,Ndirs):
            for iCell in range(0,Ncells):
                spikeTimes=allSpikeTimes[iCell]
                dirSpikes=spikeTimes[spikeTimes>=pulseTimes[iCycle,iDirection,0]]
                dirSpikes=dirSpikes[dirSpikes<pulseTimes[iCycle,iDirection,-1]+periodDuration]
                allSpikeCounts[iCell,iCycle,iDirection]=np.size(dirSpikes)
    ### END CODE HERE ###
    
    return allSpikeCounts

In [ ]:
allSpikeCounts=calculateAllSpikeCounts(pulseTimes,allSpikeTrains)
print(allSpikeCounts[3,:,:])

Expected output:

Ok, now use the ```np.mean``` function in the right dimension, to get all the tuning curves!

In [ ]:
allTuningCurves=np.mean(allSpikeCounts,axis=1)

By running the following cell, you can examine the tuning curve of the second cell. Change the indices to examine the tuning curves for different cells. Can you understand what all of them mean?

In [ ]:
plotTuningCurvePolar(angles,allTuningCurves[13,:])

## 2 - Quantification of direction selectivity

### The direction selectivity index (DSI)

The direction selectivity index (DSI) is a common quantification of direction tuning. One of the ways to calculate it is the following:

$$ DSI = \frac{1}{\sum_{k=1}^{N}{r_{k}}} \left|\sum_{k=1}^{N}{r_{k}e^{i\phi_{k}}}\right|$$

**Exercise:** Fill in ```calculateDSI(angles,responses)```. To help you, a complex number is given as 1j, 2j, 3j... You can calculated the maginude of a complex number with ```np.absolute()```.

In [ ]:
def calculateDSI(angles,responses):
    '''
    Inputs:
            angles: np array of angles in radians
            responses: 
    '''
    
    ### START CODE HERE ### (approx. 1-2 lines)
    vsum=np.sum(responses*np.exp(1j*angles))/np.sum(responses)
    dsi=np.absolute(vsum)
    ### END CODE HERE ###

    return dsi

In [ ]:
print('DSI = ' + str(calculateDSI(angles, tuningCurve)))

Expected output: 0.31383540910149094

Good job! Now let's try to find the DSI values for all the cells we provided for you.

**Exercise:** Calculate the DSIs for all cells provided. Then run the cell below, and examine the histogram. 

*Hint:* Instead of using a for loop, you can use linear algebra to calculate the responses for all cells simultaneously!

In [ ]:
def calculateDSIall(angles,multiTuningCurve):
    '''
    Inputs:
            angles: np array of angles in radians
            multiTuningCurve: Ncells x Nangles np array
    Outputs:
            dsiAll: Ncells x 1 np array of dsi values for each cell
    '''
    
    ### START CODE HERE ### (approx. 1-2 lines)
    vsums=(multiTuningCurve@np.exp(1j*angles))/np.sum(multiTuningCurve,axis=1)
    dsiAll=np.absolute(vsums)
    ### END CODE HERE ###

    return dsiAll

In [ ]:
plt.hist(calculateDSIall(angles,allTuningCurves),8);

### Monte Carlo permutation for creating DSI confidence intervals

Examine the tuning curve of the following cell. What about its DSI value?

In [ ]:
plotTuningCurvePolar(angles,allTuningCurves[11,:])
print('DSI is ' + str(calculateDSI(angles,allTuningCurves[11,:])))

Although the DSI value is above 0.2, the cell barely responded to the stimulus

**Exercise:** Calculate a permutation distribution of dsis for cell 1. Now, the vectorized version of ```calculateDSI()``` will be definitely useful!

In [ ]:
spikeCounts=allSpikeCounts[11,:,:]


In [ ]:
def calculatePval(angles,spikeCounts,Npermutations):
    '''
    Inputs:
            angles: np array of angles in radians
            multiTuningCurve: Ncells x Nangles np array
    Outputs:
            dsiAll: Ncells x 1 np array of dsi values for each cell
    '''
    
    ### START CODE HERE ### (approx. 1-2 lines)
    trueDSI=calculateDSI(angles,spikeCounts)
    
    randDSI=np.zeros(Npermutations)
    for iperm in range(Npermutations):
        randCounts=np.random.permutation(np.hstack(spikeCounts))
        randCounts=np.reshape(randCounts,np.shape(spikeCounts))
        randDSI[iperm]=calculateDSI(angles,randCounts)
    pval=1-np.sum(trueDSI>np.sort(randDSI))/Npermutations
    if pval==0:
        pval=1/Npermutations
    ### END CODE HERE ###

    return pval

In [ ]:
calculatePval(angles,allSpikeCounts[11,:,:],1000)

Calculate the p-values of all DSIs observed, and plot them versus the DSI value:

In [ ]:
def calculatePvalAll(angles,spikeCountsAll,Npermutations):
    '''
    Inputs:
            angles: np array of angles in radians
            multiTuningCurve: Ncells x Nangles np array
    Outputs:
            dsiAll: Ncells x 1 np array of dsi values for each cell
    '''
    
    ### START CODE HERE ### (approx. 1-2 lines)
    Ncells=np.size(spikeCountsAll,0)
    pvalAll=np.zeros(Ncells)
    
    for iCell in range(Ncells):
        pvalAll[iCell]=calculatePval(angles,spikeCountsAll[iCell,:,:],Npermutations)
    ### END CODE HERE ###

    return pvalAll

In [ ]:
allpvals=calculatePvalAll(angles,allSpikeCounts,1000)
alldsis=calculateDSIall(angles,allTuningCurves)
plt.plot(alldsis,np.log10(allpvals),'o')

## 3 - Comparing direction selectivity between different stimuli

The data we gave you contain gratings with different parameters as well.

In [ ]:
duration=
period=
Nangles=8
Ncycles=4

Instead of a paired t-test, that assumes normality of the underlying data, we will perform a Wilcoxon signed-rank test.

In [ ]:
stats.wilcoxon(dsi1,dsi2)